### Import the Required Libraries

In [1]:
import pandas as pd
from pandas.api.types import (is_categorical_dtype, is_datetime64_any_dtype,is_numeric_dtype, is_object_dtype,)   
    
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

from prophet import Prophet

import streamlit as st

#### Display Title of the WebApp, Process in the Excel file

In [2]:
st.title('AIPL Time Series Forecasting App')

uploaded_file = st.file_uploader("Upload the excel file (having '.xlsx' extension)")
if uploaded_file is not None:
    sheet = st.text_input("Enter Sheet name in which the required data is present.")

2024-08-16 11:21:22.600 
  command:

    streamlit run c:\Users\MU_ICT_005\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


#### Read the uploaded Excel and store it in Pandas dataframe.

In [3]:
if uploaded_file is not None:
    try:
        df_in = pd.read_excel(uploaded_file, sheet_name=sheet)
    except:
        st.warning('Please enter a valid sheet name')
        
else:
    st.warning("Upload an Excel file with '.xlsx' extension.")

if 'df_in' in globals():
    st.write(f'Shape of input dataframe is {df_in.shape[0]} rows x {df_in.shape[1]} columns')

#### Filter the Input Dataframe

In [4]:
if 'df_in' in globals():
    st.subheader("Filtering the Input Dataframe")
    
    df_fil = df_in.copy()
    modify = st.selectbox("Do you want to Filter the Dataframe?", ["No","Yes"])
    
    if modify=="Yes":                  # Filtering Streamlit dataframe code taken from tylerjrichards/st-filter-dataframe repository on GitHub 

        # Try to convert datetimes into a standard format (datetime, no timezone)
        for col in df_fil.columns:
            if is_object_dtype(df_fil[col]):
                try:
                    df_fil[col] = pd.to_datetime(df_fil[col])
                except Exception:
                    pass

            if is_datetime64_any_dtype(df_fil[col]):
                df_fil[col] = df_fil[col].dt.tz_localize(None)

        modification_container = st.container()

        with modification_container:
            to_filter_columns = st.multiselect("Filter dataframe on", df_fil.columns)
            for column in to_filter_columns:
                left, right = st.columns((1, 20))
                left.write("↳")
                # Treat columns with < 10 unique values as categorical
                if is_categorical_dtype(df_fil[column]) or df_fil[column].nunique() < 10:
                    user_cat_input = right.multiselect(
                        f"Values for {column}",
                        df_fil[column].unique(),
                        default=list(df_fil[column].unique()),
                    )
                    df_fil = df_fil[df_fil[column].isin(user_cat_input)]
                elif is_numeric_dtype(df_fil[column]):
                    _min = float(df_fil[column].min())
                    _max = float(df_fil[column].max())
                    step = (_max - _min) / 100
                    user_num_input = right.slider(
                        f"Values for {column}",
                        _min,
                        _max,
                        (_min, _max),
                        step=step,
                    )
                    df_fil = df_fil[df_fil[column].between(*user_num_input)]
                elif is_datetime64_any_dtype(df_fil[column]):
                    user_date_input = right.date_input(
                        f"Values for {column}",
                        value=(
                            df_fil[column].min(),
                            df_fil[column].max(),
                        ),
                    )
                    if len(user_date_input) == 2:
                        user_date_input = tuple(map(pd.to_datetime, user_date_input))
                        start_date, end_date = user_date_input
                        df_fil = df_fil.loc[df_fil[column].between(start_date, end_date)]
                else:
                    user_text_input = right.text_input(
                        f"Text string in {column}",
                    )
                    if user_text_input:
                        df_fil = df_fil[df_fil[column].str.contains(user_text_input)]                

#### Choose the datetime column using Streamlit selectbox

In [5]:
if 'df_fil' in globals():
    st.subheader('Selecting the Time Series')
    date_col = st.selectbox("Choose the Datetime column of the dataframe", list(df_fil.columns) )

    if ('date_col' in globals()) and (date_col is not None):
        try:
            df_fil[date_col] = pd.to_datetime(df_fil[date_col])            # convert into datetime type
            ds = df_fil[date_col]
        except:
            st.warning('Column chosen is not a datetime column. Please choose a datetime column') 

#### Choose the Time Series, after column selection and if required, aggregation

In [6]:
if 'df_fil' in globals() and df_fil[date_col].dtype=='datetime64[ns]':
    choice1 = st.selectbox("Do you want to Forecast the 'Daily'/'Monthly'/'Yearly' Count or Values of a Dataframe column?", ['Count', 'Dataframe Column Values'])
    
    if choice1=='Count':
        choice3 = st.selectbox("Choose the Period for the count, which should be forecasted.", ['Monthly','Yearly','Daily'] )
    elif choice1=='Dataframe Column Values':
        choice2 = st.selectbox("Choose the column which is to be Forecasted", list(df_fil.columns))
        choice3 = st.selectbox("Choose the Period for the count, which should be forecasted.", ['Monthly','Yearly','Daily'] )
        
        choice4 = st.selectbox("Select the suitable method for aggregation", ['Mean', 'Sum'])
    
    
    if choice1 == 'Count':
        df_t = df_fil.iloc[:,0]                      # any column can be selected as we just want to count the entries in each period
        df_t.index = df_fil[date_col]                # indexing dataframe with datetime column is necessary for using resample function
        
        if choice3== 'Daily':
            time_series = df_t.resample('D').count()        # Resample the df to get daily/weekly/yearly values for time series
        elif choice3=='Monthly':    
            time_series = df_t.resample('M').count()
        else:
            time_series = df_t.resample('Y').count()

    elif choice1=='Dataframe Column Values':
        df_t = df_fil[choice2]                              
        df_t.index = df_fil[date_col]
        
        try:
            if choice3== 'Daily':
                if choice4 == 'Sum':
                    time_series = df_t.resample('D').sum()
                else:
                    time_series = df_t.resample('D').mean()
            elif choice3=='Monthly':    
                if choice4 == 'Sum':
                    time_series = df_t.resample('M').sum()
                else:
                    time_series = df_t.resample('M').mean()
            else:
                if choice4 == 'Sum':
                    time_series = df_t.resample('Y').sum() 
                else:
                    time_series = df_t.resample('Y').mean()
        
            time_series = time_series.ffill()       # any NA value will be filled with the closest preceding non NA value
        
        except:
            st.warning('Please choose a Column with float or int datatype for Forecasting')

#### Removing Outliers

In [7]:
if 'time_series' in globals():
    st.subheader("Removing Outliers")
    outlier_choice = st.selectbox('Do you want to remove outliers, if present?', ['No','Yes'])
    
    if outlier_choice=="Yes":
        # Calculate the upper and lower limits
        Q1 = time_series.quantile(0.25)
        Q3 = time_series.quantile(0.75)
        IQR = Q3 - Q1
        
        lower_lim = float(Q1 - 1.5*IQR)
        upper_lim = float(Q3 + 1.5*IQR)
        
        # Create arrays of Boolean values indicating the outlier rows
        outlier_indices_upp = np.where(time_series >= upper_lim)[0]
        outlier_indices_low = np.where(time_series <= lower_lim)[0]
        
        # Replace the Outliers
        time_series = time_series.replace(to_replace= list(time_series[outlier_indices_upp]), value= upper_lim)
        time_series = time_series.replace(to_replace= list(time_series[outlier_indices_low]), value = lower_lim)
        
        st.write('Outliers removed!')

#### Confirm Time Series choice

In [8]:
if 'time_series' in globals():
    st.subheader('Confirming Time Series')
    confirm_choice = st.selectbox('Is this the data you want Forecasted?', ['No','Yes'])

## Plot the Time Series

In [21]:
if 'confirm_choice' in globals() and confirm_choice=='Yes':     
    st.header('Time Series Graph')
    
    # Assigning y label
    if choice1=='Count':
        ylab = 'Count'
    else:
        ylab = choice2
    
    fig1 = plt.figure(figsize=(8,4))
    plt.plot(time_series)
    plt.title("Original Time Series")
    plt.xlabel(date_col)
    plt.ylabel(ylab)
    plt.xticks(rotation=45)
    
    st.pyplot(fig1)

## Seasonal Decomposition of Time Series

In [10]:
if 'confirm_choice' in globals() and confirm_choice=='Yes':
    st.header('Seasonal Decomposition Graph')
    
    # Performing and Plotting seasonal decomposition
    stl_result = seasonal_decompose(time_series, model='additive', period=12, extrapolate_trend=2)  # Assuming a seasonal period of 12 for monthly data

    fig2, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=True, figsize=(10, 8))
    
    # Original time series
    ax1.plot(time_series.index, time_series)
    ax1.set_title('Original Time Series')
    
    # Trend component
    ax2.plot(time_series.index, stl_result.trend)
    ax2.set_title('Trend Component')
    
    # Seasonal component
    ax3.plot(time_series.index, stl_result.seasonal)
    ax3.set_title('Seasonal Component')
    
    # Residual component (noise)
    ax4.plot(time_series.index, stl_result.resid)
    ax4.set_title('Residual Component')
    plt.xticks(rotation=45)
    fig2.supxlabel(date_col)
    fig2.supylabel(ylab)
    
    st.pyplot(fig2)

## Prophet Forecasting

In [11]:
if 'confirm_choice' in globals() and confirm_choice=='Yes':
    st.header('Forecasting using Prophet Model')
    df_prophet = pd.DataFrame( data = [time_series.index, time_series]).T
    df_prophet.columns = ['ds','y']

    # Converting 'ds' from object datatype to datetime format
    df_prophet['ds'] = pd.to_datetime(df_prophet['ds'], format='%d-%m-%Y')
    
    per1 = st.slider('Please choose the number of periods for which Prophet Forecasting is to be done', min_value=0, max_value=50, value = 12)
    

#### Predict the Forecast values and then Plot it.

In [12]:
if 'confirm_choice' in globals() and confirm_choice=='Yes':
    # Initialise and Fit the Prophet model to the data
    prophet_model = Prophet()
    prophet_model.fit(df_prophet)

    # Make predictions
    if choice3== 'Daily':
        future = prophet_model.make_future_dataframe(periods=per1, freq='D')
    elif choice3=='Monthly':
        future = prophet_model.make_future_dataframe(periods=per1, freq='M')
    elif choice3=='Yearly':
        future = prophet_model.make_future_dataframe(periods=per1, freq='Y')  
    
    
    forecast1 = prophet_model.predict(future)                                # forecast1 is a Pandas Dataframe

    # Plot the forecast and the original time series
    fig3 = prophet_model.plot(forecast1, uncertainty=False, figsize = (8,6) )

    plt.plot(df_prophet['ds'],df_prophet['y'], label ='Original Time Series', color = 'orange')
    plt.legend()
    plt.title('Prophet Model Forecasting')
    plt.xlabel(date_col)
    plt.ylabel(ylab)
    plt.xticks(rotation = 45)

    st.pyplot(fig3)

In [13]:
if 'forecast1' in globals():
    st. write('If the quantity being forecasted is strictly non-negative, then in case of slightly negative predicted values, assume them to be 0.')

#### Output the Dataframe of Forecast values

In [14]:
if 'forecast1' in globals():
    df1 = pd.DataFrame(zip(forecast1['ds'][len(time_series):], forecast1['yhat'][len(time_series):]))
    df1.columns = [date_col,'Prophet Values']
    df1[date_col] = df1[date_col].dt.date           # remove the time part to keep just the date
    df1['Prophet Values'] = df1['Prophet Values'].round(3)
    
    df1.columns = [date_col,f'Prophet Values for {ylab}']
    
    st.subheader("Dataframe of Forecasted values using Prophet")
    st.dataframe(df1)

## SARIMAX Forecasting

In [15]:
if 'confirm_choice' in globals() and confirm_choice=='Yes':
    st.header('Forecasting using SARIMAX Model')
    
    per2 = st.slider('Please choose the number of periods for which SARIMAX Forecasting is to be done', min_value=0, max_value=50, value = 12)
    

In [16]:
if 'confirm_choice' in globals() and confirm_choice=='Yes':
    # Fit SARIMAX model (the optimal order hasn’t been decided and an arbitrary order is being used currently.)
    sarima_model = SARIMAX(time_series, order=(0,1,2), seasonal_order= (0,0,1,12), trend='n')
    sarima_result = sarima_model.fit()

#### Predict the Forecast values and then Plot it.

In [17]:
if 'confirm_choice' in globals() and confirm_choice=='Yes':
    # Make predictions
    forecast_steps = per2
    forecast2 = sarima_result.forecast(steps=forecast_steps)        # forecast2 is a Pandas Series
    
    fig4 = plt.figure(figsize=(8,6))
    plt.plot(time_series, label = "Original Time Series", color = 'orange')
    plt.plot(forecast2, label = "Forecast", color = 'blue')
    plt.title(f'SARIMAX Model Forecasting')
    plt.xlabel(date_col)
    plt.ylabel(ylab)
    plt.legend()
    
    st.pyplot(fig4)

In [18]:
if 'forecast2' in globals():
    st. write('If the quantity being forecasted is strictly non-negative, then in case of slightly negative predicted values, assume them to be 0.')

#### Output the Dataframe of Forecast values

In [19]:
if 'forecast2' in globals():
    df2 = pd.DataFrame(zip(forecast2.index,forecast2))
    df2.columns = [date_col,'SARIMAX Values']
    df2[date_col] = df2[date_col].dt.date                        # remove the time part to keep just the date
    df2['SARIMAX Values'] = df2['SARIMAX Values'].round(3)
    
    df2.columns = [date_col,f'SARIMAX Values for {ylab}']
    
    st.subheader("Dataframe of Forecasted values using SARIMAX")
    st.dataframe(df2)